<a href="https://colab.research.google.com/github/kurtvalcorza/kurtvalcorza/blob/main/Transcribe_Audio_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
- Upload audio mp3 file/s to Google Drive
- Change runtime type Hardware accelerator to T4 GPU
- Install Whisper and mount Google Drive

# whisper-large-v2
- [https://github.com/openai/whisper](https://github.com/openai/whisper)

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-pmb63lg_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-pmb63lg_
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define the input directory containing audio files
input_dir = '/content/drive/MyDrive/'  #@param {type: "string"}

# Iterate over audio files in the input directory
for audio_file in os.listdir(input_dir):
    if audio_file.endswith('.mp3'):  # Adjust the file extension as needed
        audio_path = os.path.join(input_dir, audio_file)

        # Transcribe the audio file
        !whisper "{audio_path}" --model large-v3 --output_dir "{input_dir}"

100%|█████████████████████████████████████| 2.88G/2.88G [00:41<00:00, 74.8MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Tagalog
[00:00.000 --> 00:03.140]  civil engineering
[00:03.140 --> 00:04.900]  electrical po
[00:04.900 --> 00:05.800]  ah electrical
[00:05.800 --> 00:07.000]  oh wow
[00:07.000 --> 00:09.140]  kay Sir Pavi ko po ako nag-take
[00:09.140 --> 00:11.960]  dapat mamashall kayo ulit sa UPL
[00:11.960 --> 00:13.740]  pinaka-miss yung lugar ano
[00:13.740 --> 00:15.660]  ay malimit po yan
[00:15.660 --> 00:17.520]  kasi meron silang ano
[00:17.520 --> 00:19.760]  meron po silang MOU
[00:19.760 --> 00:20.680]  with
[00:20.680 --> 00:23.000]  Institute of Biology
[00:23.000 --> 00:24.060]  tama ba John?
[00:24.140 --> 00:24.760]  ay ano po
[00:24.760 --> 00:27.420]  Museum of Natural History
[00:27.420 --> 00:28.820]  yes po
[00:30.000 --> 00:33.400]  nakita ko sa ano
[00:33.400 --> 00:35.000]  sa proj

# whisper-large-v3
- https://huggingface.co/openai/whisper-large-v3

In [ ]:
# Install necessary libraries
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 1: Imports
import os
import torch
import librosa
import numpy as np
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from tqdm import tqdm

# STEP 2: Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"\nDevice set to use {device}")

# STEP 3: Load model & processor
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# STEP 4: Define parameters
chunk_duration = 30.0  # seconds
generate_kwargs = {
    "max_length": 448,
    "return_timestamps": "word",  # optional: "word" or "segment"
    "do_sample": False,
}

# STEP 5: Define input directory
input_dir = '/content/drive/MyDrive/'  #@param {type: "string"}

# STEP 6: Find audio files
audio_files = [
    os.path.join(input_dir, f)
    for f in os.listdir(input_dir)
    if f.endswith('.mp3') or f.endswith('.wav')
]

# STEP 7: Transcribe files
for audio_path in tqdm(audio_files, desc="Transcribing audio files"):
    print(f"\n🗂️ Transcribing: {audio_path}")

    # Load audio
    audio_array, sr = librosa.load(audio_path, sr=16000)
    total_duration = librosa.get_duration(y=audio_array, sr=sr)
    num_chunks = int(np.ceil(total_duration / chunk_duration))

    print(f"📏 Total duration: {total_duration:.2f} sec → {num_chunks} chunks of {chunk_duration:.0f} sec")

    full_transcript = ""

    for i in tqdm(range(num_chunks), desc="Chunks", leave=False):
        start_sample = int(i * chunk_duration * sr)
        end_sample = int(min((i + 1) * chunk_duration * sr, len(audio_array)))
        chunk = audio_array[start_sample:end_sample]

        # Process this chunk
        inputs = processor(chunk, sampling_rate=16000, return_tensors="pt")
        input_features = inputs.input_features.to(device, dtype=torch_dtype)

        # Generate
        generated_tokens = model.generate(input_features=input_features, **generate_kwargs)
        decoded = processor.batch_decode(generated_tokens, skip_special_tokens=True)[0].strip()

        # Print live chunk result
        start_time = i * chunk_duration
        end_time = min((i + 1) * chunk_duration, total_duration)
        print(f"[{start_time:06.2f} --> {end_time:06.2f}] {decoded}")

        # Append to full transcript
        full_transcript += f"[{start_time:.2f} --> {end_time:.2f}] {decoded}\n"

    # Save transcript to .txt
    transcript_path = audio_path + ".txt"
    with open(transcript_path, "w", encoding="utf-8") as f:
        f.write(full_transcript)
    print(f"\n✅ Saved transcript to: {transcript_path}\n")


Device set to use cuda:0


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

[4380.00 --> 4410.00] near the end, AI isn't just another general purpose technology like, say, electricity. It's evolving into something more. A gatekeeper technology. Gatekeeper. There's that phrase again. Yeah. Meaning it uniquely holds the key, potentially, both to exponential advancements in human well-being and simultaneously to significant, maybe even existential, risks for society's future. It controls access to so many potential futures. So it really leaves us as a species with.



Chunks:  99%|█████████▉| 148/149 [10:09<00:04,  4.00s/it]

[4410.00 --> 4440.00] a fundamental choice. The guide, as you said, has offered one way to move forward, one framework. It suggests that responsible innovation comes from deliberately prioritizing ethical purposes and values. It's about consciously trying to steer the course of our algorithmic creations in accordance with a shared vision of what a better human future should look like, not just letting the tech lead us wherever it may go. Ultimately, the guide reminds us, and this is a powerful closing thought, that it is humankind



Transcribing audio files: 100%|██████████| 1/1 [10:31<00:00, 631.87s/it]

[4440.00 --> 4455.84] that must ultimately choose which direction the key will turn. The choice is ours. The technology doesn't choose for us. That really is a provocative thought to leave you, our listeners, with. How will you, in your own way, contribute to shaping this future? Which direction will you help turn that key?

✅ Saved transcript to: /content/drive/MyDrive/AI Governance.wav.txt

